# patent와 compustat merge

### 할 일
1. patent의 assignee name에서 plc, kft 제거
2. patent와 compustat assignee name, year 매칭 성공률 확인
3. assignee name, year 기준으로 matching

### 1. patent의 assignee name에서 plc, kft 제거

In [1]:
import pandas as pd
import os

os.chdir("E:/patent")

In [8]:
def noplc(s):
    
    rm=['inc','ltd','llc','pte','plc','kft']
    for i in range(0,len(s)):
        if s[-3:] in rm:
            s=s[:-3]
        else:
            break

    return s

In [6]:
df =  pd.read_csv('patents/pid_assignee.csv')
list(df)

C:\Users\DW\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['patent_id', 'new_assignee', 'assignee']

In [9]:
for i in range(0,len(df.index)):
    s = str(df.iloc[i,1])
    a = ['plc','kft']
    
    if s[-3:] in a:
        df.iloc[i,1] = noplc(df.iloc[i,1])

In [11]:
df.to_csv('pid_assignee_2.csv',index=False)

### 2. patent와 compustat assignee name 매칭 성공률 확인

pid에 있는 new assignee list와 compustat의 new assignee list 비교

In [20]:
df = pd.read_csv('pid_assignee_date_deg.csv')
pid = df['new_assignee']
pid = pid.drop_duplicates().tolist()
len(pid)

119703

In [21]:
pid.sort()
pid[0:10]

['!j',
 '!maginethisrenovations',
 '&angstmsystemab',
 '&cokg',
 '&commatsecuritybroadbandcorp',
 '&commatyourcommand',
 '&ldquoalesa&rdquo&ldquolalliagilegeretsesapplications&rdquo',
 '&ldquodurable&rdquohunkejochheimcokg',
 '&oumlhlinsracingab',
 '&primebots']

In [12]:
df = pd.read_csv('compustat/compustat.csv')
comp = df['new_assignee']
comp = comp.drop_duplicates().tolist()
len(comp)

71098

In [16]:
comp[0:10]

['01cyberatonsa',
 '1-800-attorney',
 '1-800-flowerscom',
 '1-800contacts',
 '1-page',
 '1000mercis',
 '1001opticalco',
 '1010printinggroup',
 '104corp',
 '10taclestudiosag']

In [22]:
del df

In [28]:
match = list(set(pid).intersection(comp))
len(match)

4192

In [29]:
match[0:10]

['sktelecomco',
 'minoceramicco',
 'daidoconcreteco',
 'takamisawaelectricco',
 'webmethods',
 'tennesseevalleyauthority',
 'softingag',
 'nipponkatanco',
 'newportnewsshipbuilding',
 'iharachemicalindustryco']

In [17]:
import csv

with open('patent_comp_matched_list.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(ass)

### 2-1. assignee + year matching 성공률 확인

year까지 넣어서 얼마나 matching 가능한가 봐야지

In [3]:
df = pd.read_csv('pid_assignee_date_deg.csv')
pid = df[['patent_id','new_assignee','year']]
len(pid)

1485282

In [7]:
df = pd.read_csv('compustat/compustat.csv')
comp = df[['new_assignee','fyear','revt']]
len(comp)

851180

In [11]:
print type(pid.iloc[1,2]), type(comp.iloc[1,1])

<type 'numpy.float64'> <type 'numpy.float64'>


In [12]:
del df
comp.rename(columns={'fyear': 'year'}, inplace=True)
list(comp)

['new_assignee', 'year', 'revt']

In [13]:
match = pid.merge(comp, how='inner', on=['new_assignee','year'])
len(match)

264559

In [14]:
match.iloc[0:10,]

,patent_id,new_assignee,year,revt
0,4763686,halliburtonco,1988.0,4825.7
1,4790341,halliburtonco,1988.0,4825.7
2,4791952,halliburtonco,1988.0,4825.7
3,4793411,halliburtonco,1988.0,4825.7
4,4817437,halliburtonco,1988.0,4825.7
5,4823594,halliburtonco,1988.0,4825.7
6,4829811,halliburtonco,1988.0,4825.7
7,4846769,halliburtonco,1988.0,4825.7
8,4848145,halliburtonco,1988.0,4825.7
9,4848461,halliburtonco,1988.0,4825.7


In [18]:
match.to_csv('patent_comp_matched_sample.csv',index=False)

In [16]:
ass = match['new_assignee']
ass = ass.drop_duplicates().tolist()
len(ass)

3393

In [17]:
import csv

with open('patent_comp_matched_list.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(ass)

__정리__
- patent: 119703개 기업
- compustat: 71098개 기업
- matching: 3393개 기업   
=> 전체 patent 기업 중 __2.8%__만 financial data 확보 가능 (그러나 financial data에도 missing 많아서 실제로는 더 줄어들 듯)
  

- patent: 1485282개
- compustat: 851180개
- matching: 264559개   
=> 하지만 전체 observation 중에서는 __17.8%__ 정도 살릴 수 있음

__문제__
- 중간에 이름 바뀐 기업 추적 불가 (ex 구글의 경우 compustat에는 alphabet으로 나옴)  
- 비상장사에 대한 추적 거의 불가  
  
__=> 어쩔 수 없다 그냥 가자__

### 3. assignee name, year 기준으로 matching

In [2]:
pid = pd.read_csv('pid_assignee_date_deg.csv')
comp = pd.read_csv('compustat/compustat.csv')
print len(pid), len(comp)

C:\Users\DW\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1485282 851180


In [3]:
comp.rename(columns={'fyear': 'year'}, inplace=True)
match = pid.merge(comp, how='inner', on=['new_assignee','year'])
len(match)

264559

In [6]:
match.iloc[0:10,]

,patent_id,new_assignee,assignee,date,year,forward,backward,conm,conml,act,...,nicon,ninc,ninoc,ppegt,ppent,revt,teq,xlr,xrd,xstfws
0,4763686,halliburtonco,HALLIBURTON CO.,1988-01-22,1988.0,0.0,39.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN
1,4790341,halliburtonco,HALLIBURTON CO.,1988-01-22,1988.0,1.0,38.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN
2,4791952,halliburtonco,HALLIBURTON CO.,1988-01-22,1988.0,0.0,36.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN
3,4793411,halliburtonco,HALLIBURTON CO.,1988-06-29,1988.0,3.0,17.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN
4,4817437,halliburtonco,HALLIBURTON CO.,1988-07-08,1988.0,2.0,3.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN
5,4823594,halliburtonco,HALLIBURTON CO.,1988-03-14,1988.0,24.0,15.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN
6,4829811,halliburtonco,HALLIBURTON CO.,1988-04-08,1988.0,7.0,6.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN
7,4846769,halliburtonco,HALLIBURTON CO.,1988-02-10,1988.0,1.0,8.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN
8,4848145,halliburtonco,HALLIBURTON CO.,1988-09-21,1988.0,22.0,10.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN
9,4848461,halliburtonco,HALLIBURTON CO.,1988-06-24,1988.0,7.0,6.0,HALLIBURTON CO,Halliburton Co,NaN,...,NaN,NaN,93.6,3283.3,964.4,4825.7,2107.2,2069.4,81.0,NaN


In [7]:
match.to_csv('patent_compustat.csv',index=False)

__assignee, year 기준 sorting은 excel로 마무리__